In [28]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [2]:
data1 = pd.read_parquet('fhv_tripdata_2021-01.parquet')
data2 = pd.read_parquet('fhv_tripdata_2021-02.parquet')

In [3]:
len(data1)

1154112

In [6]:
data1['duration'] = data1.dropOff_datetime - data1.pickup_datetime
data1.duration = data1.duration.apply(lambda td: td.total_seconds() / 60)

data2['duration'] = data2.dropOff_datetime - data2.pickup_datetime
data2.duration = data2.duration.apply(lambda td: td.total_seconds() / 60)

In [7]:
data1.duration.mean()

19.1672240937939

In [8]:
data1 = data1[(data1.duration >= 1) & (data1.duration <= 60)]
data2 = data2[(data2.duration >= 1) & (data2.duration <= 60)]

In [10]:
data1.PUlocationID = data1.PUlocationID.fillna(-1)
data1.DOlocationID = data1.DOlocationID.fillna(-1)
data2.PUlocationID = data2.PUlocationID.fillna(-1)
data2.DOlocationID = data2.DOlocationID.fillna(-1)

<ipython-input-10-3ed78669103c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data1.PUlocationID = data1.PUlocationID.fillna(-1)
<ipython-input-10-3ed78669103c>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data1.DOlocationID = data1.DOlocationID.fillna(-1)


In [15]:
100.0*data1.PUlocationID.value_counts()/len(data1)

-1.0      83.527328
 221.0     0.750568
 206.0     0.612438
 129.0     0.484671
 115.0     0.367805
            ...    
 111.0     0.000451
 27.0      0.000360
 34.0      0.000270
 2.0       0.000180
 110.0     0.000090
Name: PUlocationID, Length: 262, dtype: float64

In [23]:
categorical = ['PUlocationID', 'DOlocationID']

In [47]:
data = pd.concat([data1, data2]).reset_index(drop=True)

In [48]:
data[categorical] = data[categorical].astype(str)

In [56]:
train_dicts = data[categorical].to_dict(orient='records')
dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)

In [60]:
X_test = X_train[1109826:]
X_train = X_train[:1109826]

In [62]:
X_train

<1109826x526 sparse matrix of type '<class 'numpy.float64'>'
	with 2219652 stored elements in Compressed Sparse Row format>

In [63]:
target = 'duration'
y_train = data1[target].values
y_test = data2[target].values

lr = LinearRegression()
lr.fit(X_train, y_train)

LinearRegression()

In [64]:
y_pred = lr.predict(X_train)
mean_squared_error(y_train, y_pred, squared=False)

10.528519107205586

In [65]:
X_test

<990113x526 sparse matrix of type '<class 'numpy.float64'>'
	with 1980226 stored elements in Compressed Sparse Row format>

In [66]:
y_pred = lr.predict(X_test)
mean_squared_error(y_test, y_pred, squared=False)

11.014283135824014